In [2]:
using Pkg
Pkg.activate(".")

using DataFrames
using NemoMod
using SQLite
using Cbc
using JuMP

include("setup_analysis.jl")

  Activating project at `~/Documents/Projects/git_jbus/lac_decarbonization/julia`


"/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/tmp/nemomod_intermediate_database.sqlite"

In [3]:
ispath(fp_sqlite_nemomod_db_tmp)

true

In [32]:
!ispath(dir_tmp) ? mkdir(dir_tmp) : nothing
fp_db = joinpath(dir_tmp, "testingdb.sqlite")
#createnemodb(fp_db)

2022-14-Aug 13:49:56.453 Opened SQLite database at /Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/tmp/testingdb.sqlite.
2022-14-Aug 13:49:56.684 Added NEMO structure to SQLite database at /Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/tmp/testingdb.sqlite.


SQLite.DB("/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/tmp/testingdb.sqlite")

In [31]:
rm(fp_db)
createnemodb(fp_db)

2022-14-Aug 13:18:02.188 Opened SQLite database at /Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/tmp/testingdb.sqlite.
2022-14-Aug 13:18:02.267 Added NEMO structure to SQLite database at /Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/tmp/testingdb.sqlite.


SQLite.DB("/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/tmp/testingdb.sqlite")

In [10]:

db2 = SQLite.DB(fp_db)

dict_tables2 = Dict{String, Dat aFrame}()
tables_vary2 = []

tabs = SQLite.tables(db2)[1]
table_name = 0
for k in 1:length(tabs)
    table_name = tabs[k]
    df_tmp = DBInterface.execute(db2, "select * from $(table_name)") |> DataFrame
    dict_tables2[table_name] = df_tmp
    (nrow(df_tmp) > 0) ? push!(tables_vary2, table_name) : nothing
end



In [29]:
df_out = DBInterface.execute(db2, "select sql from sqlite_master where type = 'index';")



SQLite.Query(SQLite.Stmt(SQLite.DB("/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/tmp/testingdb.sqlite"), 94), Base.RefValue{Int32}(100), [:sql], Type[Union{Missing, String}], Dict(:sql => 1), Base.RefValue{Int64}(0))

### Try running the model

In [11]:
@time NemoMod.calculatescenario(
    fp_sqlite_nemomod_db_tmp; 
    jumpmodel = Model(Cbc.Optimizer),
    numprocs = 1
)

2022-14-Aug 18:47:37.208 Started modeling scenario.
2022-14-Aug 18:47:37.215 Validated run-time arguments.
2022-14-Aug 18:47:37.224 Connected to scenario database. Path = /Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/tmp/nemomod_intermediate_database.sqlite.
2022-14-Aug 18:47:37.257 Dropped pre-existing result tables from database.
2022-14-Aug 18:47:37.258 Verified that transmission modeling is not enabled.
2022-14-Aug 18:47:37.366 Created parameter views and indices.
2022-14-Aug 18:47:37.380 Created temporary tables.
2022-14-Aug 18:47:37.680 Executed core database queries.
2022-14-Aug 18:47:37.692 Defined dimensions.
2022-14-Aug 18:47:37.696 Defined demand variables.
2022-14-Aug 18:47:37.702 Defined storage variables.
2022-14-Aug 18:47:37.703 Defined capacity variables.
2022-14-Aug 18:47:37.879 Defined activity variables.
2022-14-Aug 18:47:37.880 Defined costing variables.
2022-14-Aug 18:47:37.880 Defined reserve margin variables.
2022-14-Aug 18:47:37.882 Defined emissi

INFEASIBLE_OR_UNBOUNDED::TerminationStatusCode = 6

In [64]:
filter(x -> occursin("Emission", x), keys(dict_tables_demo))

Set{String} with 6 elements:
  "ModelPeriodEmissionLimit"
  "ModelPeriodExogenousEmission"
  "AnnualEmissionLimit"
  "EmissionsPenalty"
  "AnnualExogenousEmission"
  "EmissionActivityRatio"

In [119]:
using GLPK
using Clp


In [126]:
dbpath = normpath(joinpath(pathof(NemoMod), "..", "..", "test", "storage_test.sqlite"))
chmod(dbpath, 0o777)
t0 = time()
NemoMod.calculatescenario(dbpath; jumpmodel=Model(Cbc.Optimizer), numprocs = 1)
t1 = time()

2022-17-Aug 20:20:24.818 Started modeling scenario.
2022-17-Aug 20:20:24.819 Validated run-time arguments.
2022-17-Aug 20:20:24.823 Connected to scenario database. Path = /Users/jsyme/.julia/packages/NemoMod/vg6Tx/test/storage_test.sqlite.
2022-17-Aug 20:20:24.838 Dropped pre-existing result tables from database.
2022-17-Aug 20:20:24.838 Verified that transmission modeling is not enabled.
2022-17-Aug 20:20:24.885 Created parameter views and indices.
2022-17-Aug 20:20:24.894 Created temporary tables.
2022-17-Aug 20:20:25.190 Executed core database queries.
2022-17-Aug 20:20:25.193 Defined dimensions.
2022-17-Aug 20:20:25.494 Defined demand variables.
2022-17-Aug 20:20:25.495 Defined storage variables.
2022-17-Aug 20:20:25.495 Defined capacity variables.
2022-17-Aug 20:20:25.651 Defined activity variables.
2022-17-Aug 20:20:25.676 Defined costing variables.
2022-17-Aug 20:20:25.677 Defined reserve margin variables.
2022-17-Aug 20:20:25.677 Defined emissions variables.
2022-17-Aug 20:20:2

1.660792828169395e9

In [128]:
t1 - t0

3.368569850921631

In [74]:
# get tables from SISEPUEDE
db = SQLite.DB(fp_sqlite_nemomod_db_tmp)

dict_tables = Dict{String, DataFrame}()
tables_vary = []

tabs = SQLite.tables(db)[1]
table_name = 0
for k in 1:length(tabs)
    table_name = tabs[k]
    df_tmp = DBInterface.execute(db, "select * from $(table_name)") |> DataFrame
    dict_tables[table_name] = df_tmp
    (nrow(df_tmp) > 0) ? push!(tables_vary, table_name) : nothing
end


# get tables from DEMO
db_demo = SQLite.DB(dbpath)

dict_tables_demo = Dict{String, DataFrame}()
tables_vary_demo = []

tabs_demo = SQLite.tables(db_demo)[1]
for k in 1:length(tabs_demo)
    table_name = tabs_demo[k]
    df_tmp = DBInterface.execute(db_demo, "select * from $(table_name)") |> DataFrame
    dict_tables_demo[table_name] = df_tmp
    (nrow(df_tmp) > 0) ? push!(tables_vary_demo, table_name) : nothing
end

In [10]:
#setdiff(keys(dict_tables_demo), keys(dict_tables))
for k in keys(dict_tables_demo)
    #(("f" in names(dict_tables_demo[k])) & (nrow(dict_tables_demo[k]) > 0)) ? println(k) : nothing
    if k in keys(dict_tables)
        if (nrow(dict_tables_demo[k]) != 0) & (nrow(dict_tables[k]) == 0)
            println(k)
        end
    end
        
end
#dict_tables_demo["InputActivityRatio"]

StorageLevelStart
StorageFullLoadHours


In [19]:
unique(dict_tables_demo["OutputActivityRatio"][:, :f])

filter(x -> (x[:y] == "2025"), dict_tables_demo["OutputActivityRatio"])

,id,r,t,f,m,y,val
,Int64,String,String,String,String,String,Float64
1,6,1,gas,electricity,generate,2025,1.0
2,16,1,solar,electricity,generate,2025,1.0
3,26,1,storage1,electricity,generate,2025,1.0
4,36,1,gassupply,gas,generate,2025,1.0
5,46,1,solarsupply,solar,generate,2025,1.0


In [47]:
for k in keys(dict_tables_demo)
    
    if "t" in names(dict_tables_demo[k])
        (("gasssupply" in dict_tables_demo[k][:, :t]) | ("solarsupply" in dict_tables_demo[k][:, :t])) ? println(k) : nothing
    end
end

OperationalLife
OutputActivityRatio
vproductionbytechnologyannual
vnewcapacity
vtotalcapacityannual


In [80]:
unique(dict_tables_demo["OutputActivityRatio"][:, :t])

5-element Vector{String}:
 "gas"
 "solar"
 "storage1"
 "gassupply"
 "solarsupply"

In [86]:
filter(z -> (z[:t] == "gas"), dict_tables_demo["OutputActivityRatio"])

,id,r,t,f,m,y,val
,Int64,String,String,String,String,String,Float64
1,1,1,gas,electricity,generate,2020,1.0
2,2,1,gas,electricity,generate,2021,1.0
3,3,1,gas,electricity,generate,2022,1.0
4,4,1,gas,electricity,generate,2023,1.0
5,5,1,gas,electricity,generate,2024,1.0
6,6,1,gas,electricity,generate,2025,1.0
7,7,1,gas,electricity,generate,2026,1.0
8,8,1,gas,electricity,generate,2027,1.0
9,9,1,gas,electricity,generate,2028,1.0


In [92]:
filter(z -> (z[:t] == "gas"), dict_tables_demo["InputActivityRatio"])



,id,r,t,f,m,y,val
,Int64,String,String,String,String,String,Float64
1,1,1,gas,gas,generate,2020,2.0
2,2,1,gas,gas,generate,2021,2.0
3,3,1,gas,gas,generate,2022,2.0
4,4,1,gas,gas,generate,2023,2.0
5,5,1,gas,gas,generate,2024,2.0
6,6,1,gas,gas,generate,2025,2.0
7,7,1,gas,gas,generate,2026,2.0
8,8,1,gas,gas,generate,2027,2.0
9,9,1,gas,gas,generate,2028,2.0


In [107]:
filter(x -> (x[:t] == "gassupply"), dict_tables_demo["OutputActivityRatio"])




,id,r,t,f,m,y,val
,Int64,String,String,String,String,String,Float64
1,31,1,gassupply,gas,generate,2020,1.0
2,32,1,gassupply,gas,generate,2021,1.0
3,33,1,gassupply,gas,generate,2022,1.0
4,34,1,gassupply,gas,generate,2023,1.0
5,35,1,gassupply,gas,generate,2024,1.0
6,36,1,gassupply,gas,generate,2025,1.0
7,37,1,gassupply,gas,generate,2026,1.0
8,38,1,gassupply,gas,generate,2027,1.0
9,39,1,gassupply,gas,generate,2028,1.0


In [108]:
for k in keys(dict_tables_demo)
    if "t" in names(dict_tables_demo[k])
        if "gassupply" in dict_tables_demo[k][:, :t]
            println(k)
        end
    end
    
    if "val" in names(dict_tables_demo[k])
        if "gassupply" in dict_tables_demo[k][:, :val]
            println(k)
        end
    end
    
    if "f" in names(dict_tables_demo[k])
        if "gassupply" in dict_tables_demo[k][:, :f]
            println(k)
        end
    end
end

VariableCost
OperationalLife
OutputActivityRatio
vproductionbytechnologyannual
TECHNOLOGY
vnewcapacity
vtotalcapacityannual


In [109]:
for k in keys(dict_tables_demo)
    if "f" in names(dict_tables_demo[k])
        if "gas" in dict_tables_demo[k][:, :f]
            println(k)
        end
    end
    
    if "val" in names(dict_tables_demo[k])
        if "gas" in dict_tables_demo[k][:, :val]
            println(k)
        end
    end
    
   
end

vusenn
vdemandnn
FUEL
vproductionnn
OutputActivityRatio
vusebytechnologyannual
vproductionbytechnologyannual
TECHNOLOGY
InputActivityRatio


In [110]:
filter(x -> (x[:f] == "gas"), dict_tables_demo["InputActivityRatio"])

,id,r,t,f,m,y,val
,Int64,String,String,String,String,String,Float64
1,1,1,gas,gas,generate,2020,2.0
2,2,1,gas,gas,generate,2021,2.0
3,3,1,gas,gas,generate,2022,2.0
4,4,1,gas,gas,generate,2023,2.0
5,5,1,gas,gas,generate,2024,2.0
6,6,1,gas,gas,generate,2025,2.0
7,7,1,gas,gas,generate,2026,2.0
8,8,1,gas,gas,generate,2027,2.0
9,9,1,gas,gas,generate,2028,2.0


In [102]:
filter(x -> (x[:t] == "gassupply"), dict_tables_demo["OperationalLife"])

,id,r,t,val
,Int64,String,String,Float64
1,4,1,gassupply,100.0
